# 序列的修改、散列和切片

## 我们将一次性实现整个vector类，使其：
- 与vector2d类兼容
- 构造方法可接受任意个参数，最好接受可迭代的对象作为参数
- 支持切片
- 实现散列
- 将格式化显示从二维提升到多维
- 通过特殊属性访问向量前几个分量的值

In [28]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode,
                                 components)  # 受保护的实例属性，保存Vector的分量

    def __iter__(self):
        return iter(self._components)  # 构建迭代器

    def __repr__(self):
        components = reprlib.repr(
            self._components)  # 获取self._components的有限长度表现形式
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (
            bytes([ord(self.typecode)]) + bytes(self._components))  #构建bytes对象

    def __eq__(self, other):  # 使用all方法来实现eq
        return (len(self) == len(other)
                and all(a == b for a, b in zip(self, other)))

    def __hash__(self):  #实现散列函数，使用reduce来实现累计异或
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes,
                                0)  # reduce函数最好设置第三个参数，即初始值，避免出现空序列处理错误

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):  # 鸭子类型，以支持序列操作
        return len(self._components)

    def __getitem__(self, index):  # 鸭子类型，以支持序列操作
        cls = type(self)
        if isinstance(index, slice):  #为了保证得到的切片也是Vector类
            return cls(self._components[index]
                       )  #调用类的构造方法，使用_components数组的切片构建一个新的Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):  #通过特殊句法读取前四个分量，__attr__方法能够在查找属性失败之后处理参数
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):  #将Vector2d中的二维极坐标表示扩展到多维
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  #把memoryview传给构造方法

## 协议和鸭子类型
- 在Python中创建功能完善的序列类型无需使用继承，只需实现符合序列协议的方法
- 协议时是正式的接口，只在文档中定义，在代码中不定义
- 例如，Python的序列协议只需要```__len__```和```__getitem__```两个方法。任何类只要使用标准的签名和语义实现了这两个方法，就能用在任何期待序列的地方
- 我们称其为序列，是因为其行为像序列，人们称其为鸭子类型

## 切片原理
- 查看切片的index具体是如何表示的

In [29]:
class MySeq:
    def __getitem__(self, index):
        return index

In [30]:
s = MySeq()
s[1]

1

In [31]:
s[1:4]

slice(1, 4, None)

In [32]:
s[1:4:2]

slice(1, 4, 2)

In [33]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [34]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

## indices属性
- s.indices(len) -> (start,stop,stride)
- 一个序列的indices属性实现了在给定长度len下，选定的切片的表示

In [35]:
slice(None, 10, 2).indices(7)

(0, 7, 2)

In [36]:
slice(-3, -7, -2).indices(100)

(97, 93, -2)